## Logistic Regression

Breast Cancer data from [the UCI repository](http://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29) contains records corresponding to 
cases of observed tumors.   There are a number of observations for each and a categorisation in the `class` column: 2 for benign (good), 4 for malignant (bad).  Your task is to build a logistic regression model to classify these cases. 

The data is provided as a CSV file.  There are a small number of cases where no value is available, these are indicated in the data with `?`. I have used the `na_values` keyword for `read_csv` to have these interpreted as `NaN` (Not a Number).  Your first task is to decide what to do with these rows. You could just drop these rows or you could [impute them from the other data](http://scikit-learn.org/stable/modules/preprocessing.html#imputation-of-missing-values).

You then need to follow the procedure outlined in the lecture for generating a train/test set, building and evaluating a model. Your goal is to build the best model possible over this data.   Your first step should be to build a logistic regression model using all of the features that are available.
  

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.feature_selection import RFE

In [2]:
bcancer = pd.read_csv("files/breast-cancer-wisconsin.csv", na_values="?")
bcancer.head()

,sample_code_number,clump_thickness,uniformity_cell_size,uniformity_cell_shape,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1.0,3,1,1,2
1,1002945,5,4,4,5,7,10.0,3,2,1,2
2,1015425,3,1,1,1,2,2.0,3,1,1,2
3,1016277,6,8,8,1,3,4.0,3,7,1,2
4,1017023,4,1,1,3,2,1.0,3,1,1,2


In [4]:
# Examine the data, look at the statistical summary etc.
bcancer.isnull().sum() ## Use this to find null values by row
bcancer = bcancer.dropna() ## Use this to drop null values

In [8]:
# deal with the NaN values in the data
bcancer.isnull().sum()
print(bcancer.shape)

(683, 11)


In [24]:
# Build your first model - defining training and test data sets then use Logistic Regression to build a model
data = bcancer[['clump_thickness', 'uniformity_cell_size', 'uniformity_cell_shape', 'marginal_adhesion',
           'single_epithelial_cell_size', 'bare_nuclei', 'bland_chromatin', 'normal_nucleoli', 'mitoses']]
label = bcancer['class']
print("Data shape: " + str(data.shape))
print("Label shape: " + str(label.shape))

X_train, X_test, Y_train, Y_test = train_test_split(data, label, random_state=42, test_size=0.2)
print(X_test.shape)
print(X_train.shape)
print(Y_test.shape)
print(Y_train.shape)

ls = LogisticRegression()
ls.fit(X_train, Y_train)
Y_pred = ls.predict(X_test)

Data shape: (683, 9)
Label shape: (683,)
(137, 9)
(546, 9)
(137,)
(546,)


### Evaluation

To evaluate a classification model we want to look at how many cases were correctly classified and how many
were in error.  In this case we have two outcomes - benign and malignant.   SKlearn has some useful tools, the 
[accuracy_score]() function gives a score from 0-1 for the proportion correct.  The 
[confusion_matrix](http://scikit-learn.org/stable/modules/model_evaluation.html#confusion-matrix) function 
shows how many were classified correctly and what errors were made.  Use these to summarise the performance of 
your model (these functions have already been imported above).

In [30]:
# Evaluate the model

print(accuracy_score(Y_test, Y_pred))
print(confusion_matrix(Y_test, Y_pred))

0.948905109489051
[[78  1]
 [ 6 52]]


### Feature Selection

Since you have many features available, one part of building the best model will be to select which features to use as input to the classifier. Your initial model used all of the features but it is possible that a better model can 
be built by leaving some of them out.   Test this by building a few models with subsets of the features - how do your models perform? 

This process can be automated.  The [sklearn RFE function](http://scikit-learn.org/stable/modules/feature_selection.html#recursive-feature-elimination) implements __Recursive Feature Estimation__ which removes 
features one by one, evaluating the model each time and selecting the best model for a target number of features.  Use RFE to select features for a model with 3, 4 and 5 features - can you build a model that is as good or better than your initial model?

In [54]:
from sklearn.svm import SVR

estimator = SVR(kernel='linear')
selector = RFE(estimator, 1, step=1)
selector = selector.fit(X_train, Y_train)
Y_pred = selector.predict(X_test)

n = 0
while n < len(Y_pred):
    Y_pred[n] = round(Y_pred[n])
    Y_pred[n] = int(Y_pred[n])
    n = n+1

print(accuracy_score(Y_test, Y_pred))

0.7883211678832117


## Conclusion

Write a brief conclusion to your experiment.  You might comment on the proportion of __false positive__ and __false negative__ classifications your model makes.  How useful would this model be in a clinical diagnostic setting? 